In [8]:
# Tratamiento de datos
import numpy as np
import pandas as pd


# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

from scipy import stats

# Preprocesado y modelado
plt.rcParams["figure.figsize"] = (10,8)

In [9]:
# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

# Input
Cargamos los datos:

In [ ]:
df = pd.read_csv("data/insurance.csv")
df.head()

# Análisis exploratorio

1. ver que tengo en el df y los tipos

In [ ]:
df.info()

- variables continuas:
  - `charges`
  - `bmi`
  - `age`
- variables discretas:
  - `sex`
  - `children`
  - `region`


la variable respuesta -- la que se va a intentar predecir. en este caso, será `charges`. el resto de variables serán varables predictoras:
- `age`
- `sex`
- `bmi`
- `children`
- `smoker`
- `region`

2. ver el número de nulos que tengo en el dataset y ver que puedo hacer con ello

una columna es quitable en cuanto a úmero de nulos será el 60%-70% de valores nulos

In [ ]:
# Número de datos nulos por columna
# ==============================================================================
df.isnull().sum() / df.shape[0]

3. check duplicados -- si hay filas duplicadas, te las zumbas

In [ ]:
df.duplicated().sum() / df.shape[0]

4. check los tipos de datos

In [ ]:
df.describe().T

In [ ]:
sns.pairplot(data=df);

## Variable respuesta

la variable respuesta en este caso será `charges`.

In [ ]:
# vamos a ver visualmente como está distribuida la variable respuesta

##esto es un resumen un histograma

sns.kdeplot(
    df.charges, 
    color = "blue", 
    fill = True);

In [ ]:
# cuantas casas tienen de media el valor que indica el eje x
sns.histplot(data = df, x = 'charges', kde = True);

La variable respuesta no es normal -- aunque tiene una forma normal, con cierto sesgo hacia la izquierda.

In [ ]:
stats.shapiro(df["charges"])

otra de las soluciones posibles, sin transformar la gráfica sería dividir el conjunto de datos y quitarnos la parte de la derecha, ya que es la parte que no es simétrica de la distriución.

In [ ]:
df.describe(include=np.number).T

## Variables predictoras

### Variables numéricas


In [ ]:
# exploramos cuáles son los principales estadísticos de las variables numéricas

df_numericas = df.select_dtypes(include = np.number)#.describe()
df_numericas

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize = (30,10))
axes = axes.flat
columnas = df_numericas.drop(['charges'], axis = 1)  
for i, columns in enumerate(columnas.columns):
    sns.regplot(data = df, x = columns, y = 'charges', ax = axes[i])
fig.tight_layout()

vale, confirmamos que la variable `children` es numérica, pero se puede considerar una variable categórica con 6 categorías.

la variable `bmi` se dispersa.

la variable `age` tiene una distribución más extraña -- parece que tiene tres partes que son lineales.

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(20,15))

for i in range(len(df_numericas.columns)):
    sns.boxplot(x=df_numericas.columns[i], data=df_numericas, ax=ax[i])
plt.tight_layout()
plt.show();

Mirando la mezcla de variables respuesta + variables predictoras, vemos que donde hay outliers es en el campo `bmi` de las variables predictoras y en la varable respuesta `charges`.

### Correlación entre variables numéricas

ojo: solo entre variables numéricas

In [ ]:
sns.heatmap(df.corr(), 
           cmap = "mako", 
           annot = True);

In [ ]:
# creamos un dataframe todo de unos de la misma forma que nuestra matriz de correlación
## creamos una matriz triangular

mask = np.triu(np.ones_like(df.corr(), dtype = np.bool))
mask


In [ ]:
sns.heatmap(df.corr(), 
           cmap = "YlGnBu", 
            mask = mask,
           annot = True,
           vmin = -1,
           vmax = 1
           );
## ojo, incluir los valores max y min para ver la escala total
## para ver si hay correlación entre dos variables, hay que ver si los valores están en torno al 0.6 - 0.7

Mirando la matriz de correlación, la variable que más influye en la variable respuesta es `age`.

### Variables categóricas

In [ ]:
df_categoricas = df.select_dtypes(include='object')
df_categoricas.head()

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (30, 10))

axes = axes.flat

for i, columna in enumerate(df_categoricas.columns):
    sns.countplot(data = df, 
                  x = columna, 
                  ax = axes[i]);

Vemos que tanto `sex` como `region`. Lo que sí que vemos que hay una diferencia significativa es en la cantidad de fumadores. Veamos como se relacionan las variables con los valores de `charges`.

vamos a sacar el precio medio por estados y ver como se distribuyen los precios:

In [ ]:
df_cate_sex = df.groupby('sex')['charges'].mean().reset_index()
df_cate_sex.head()

In [ ]:
sns.barplot(data=df_cate_sex, x = 'sex', y = 'charges');

In [ ]:
df_cate_smoker = df.groupby('smoker')['charges'].mean().reset_index()
df_cate_smoker.head()

In [ ]:
sns.barplot(data=df_cate_smoker, x = 'smoker', y = 'charges');

In [ ]:
df_cate_smoker = df.groupby('region')['charges'].mean().reset_index()
df_cate_smoker.head()

In [ ]:
sns.barplot(data=df_cate_smoker, x = 'region', y = 'charges');

Mirando el valor de la prima para las variables categóricas, la que más variabilidad tiene es `smokers`, ya que si la persona es fumadora tiene mucho más valor de `charges` que si no lo es.

# Gestión outliers

Lo que hemos visto en la observación de los datos, es que las columnas numéricas que más outliers tienen son `bmi` y la propia variable respuesta `charges`.

También es muy significativo como varía el valor de `charges` en función a la variable categórica `smokers`. Si las personas son fumadoras, parecen pagar más primas.

## hipótesis 01: quitamos los valores de `bmi` anómalos

In [ ]:
df_bmi = df.copy()
df_bmi.head()

In [ ]:
def detectar_outliers(lista_columnas, dataframe):

    dict_indices = {}

    for i in lista_columnas:
        Q1 = np.nanpercentile(dataframe[i], 25)
        Q3 = np.nanpercentile(dataframe[i], 75)

        IQR = Q3 - Q1

        outlier_step = IQR * 1.5

        outliers_value = dataframe[(dataframe[i] < Q1 - outlier_step) | (dataframe[i] > Q3 + outlier_step)]

        if outliers_value.shape[0] > 0:
            dict_indices[i] = outliers_value.index.tolist()
        else:
            #dict_indices[i] = 'sin outliers'
            pass
    return dict_indices

In [ ]:
dic_outliers = detectar_outliers(df_numericas.columns, df)
print(dic_outliers)

In [ ]:
indices_bmi = dic_outliers['bmi']
df_bmi.drop(indices_bmi, axis = 0, inplace=True)
df_bmi.head()

vamos a ver si los outlliers variarían mucho retirando los outliers

In [ ]:
df_bmi_numericas = df_bmi.select_dtypes(include=np.number)

fig, ax = plt.subplots(4, 1, figsize=(20,15))

for i in range(len(df_bmi_numericas.columns)):
    sns.boxplot(x=df_bmi_numericas.columns[i], data=df_bmi_numericas, ax=ax[i])
plt.tight_layout()
plt.show();

pese a retirando los outliers de `bmi`, sigo teniendo muchos valores anómalos de `charges`. voy a examinar las variables categóricas.

In [ ]:
df_cate_sex = df_bmi.groupby('sex')['charges'].mean().reset_index()
df_cate_sex.head()

In [ ]:
sns.barplot(data=df_cate_sex, x = 'sex', y = 'charges');

In [ ]:
df_cate_smoker = df_bmi.groupby('smoker')['charges'].mean().reset_index()
df_cate_smoker.head()

In [ ]:
sns.barplot(data=df_cate_smoker, x = 'smoker', y = 'charges');

In [ ]:
df_cate_smoker = df_bmi.groupby('region')['charges'].mean().reset_index()
df_cate_smoker.head()

In [ ]:
sns.barplot(data=df_cate_smoker, x = 'region', y = 'charges');

sigue ocurriendo lo mismo -- la media de `charges` según los `smokers` sigue siendo muy alta para los fumadores.

## hipótesis 02: fumadores o no fumadores

In [ ]:
df_smoker = df[df['smoker'] == 'yes']
df_non_smoker = df[df['smoker'] != 'yes']
df_smoker.head()

In [ ]:
df_smoker_numericas = df_smoker.select_dtypes(include=np.number)

fig, ax = plt.subplots(4, 1, figsize=(20,15))

for i in range(len(df_smoker_numericas.columns)):
    sns.boxplot(x=df_smoker_numericas.columns[i], data=df_smoker_numericas, ax=ax[i])
plt.tight_layout()
plt.show();